In [226]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from statistics import mode
import requests
import re
import time
from IPython.display import clear_output
from tqdm.notebook import tqdm
pd.set_option('display.max_rows', 200)

Getting links from the website

In [227]:
def get_list_of_files():
    """Parse the website, 
    gets the full list of file links"""
    #Webpage with links to datasets
    url = "http://web.mta.info/developers/turnstile.html"
    res=requests.get(url)
    if res.status_code == 200:
        #Creating parse tree through BeautifulSoup
        page=BeautifulSoup(res.content)        
        mydivs = page.find_all("div", {"class": "span-84 last"})
        file_list = []
        #finding links in the website content
        for div in mydivs:
            files=div.findAll('a')
        for file in files:
            file_list.append("http://web.mta.info/developers/"+file['href'])
        return file_list

Downloading data for the last 52 weeks (year)

In [228]:
def get_data(weeks_num,starting_date="221015"):
    '''Gets array of links and downloads necessary amount due to passed value'''
    links=get_list_of_files()    
    #looking for link that contains starting point
    res=[count for count, link in enumerate(links) if starting_date in link]
    links=links[res[0]:]
    #initial subset, all other will be added to it   
    df_first=pd.read_csv(links[0],sep=',')
    #looping through array of links to get the data
    for week in range(1,weeks_num):
        #getting df
        df_temp=pd.read_csv(links[week],sep=',')
        #concats downloaded part to main df
        df=pd.concat([df_temp,df_first],axis=0,ignore_index=True)
        #resets index
        df.reset_index(inplace=True,drop=True)
        df_first=df
        #prints out intermediate status
        print(f"Downloading {week} week of {weeks_num}")
        #loop makes a pause for one second to prevent website overload
        time.sleep(1)
        clear_output()
    print("Done")
    return df_first 


In [229]:
df=get_data(52)

Done


In [230]:
def fix_times(time):
    """Reformating all the times to be in intervals of 4 hours starting from 00:00:00"""
    hour = time[:2]
    minutes = time[3:5]
    new_time = round((int(hour) + int(minutes) / 60)/4) * 4

    if(new_time > 20): #If time rounds to 24 need to round down to 20
        new_time = 20
    elif(new_time < 10): #adding a 0 to the string if only one digit for formatting
        new_time = '0' + str(new_time)

    return f"{new_time}:00:00"

In [231]:
# Changing times to all be between hours 0, 4, 8, 12, 16, 20
df['TIME'] = df['TIME'].apply(lambda x: fix_times(x))


Almost 11_000_000 rows

In [232]:
df.shape

(10956784, 11)

Check for possible null values

In [233]:
df.isnull().sum()

C/A                                                                     0
UNIT                                                                    0
SCP                                                                     0
STATION                                                                 0
LINENAME                                                                0
DIVISION                                                                0
DATE                                                                    0
TIME                                                                    0
DESC                                                                    0
ENTRIES                                                                 0
EXITS                                                                   0
dtype: int64

No null values

Check for colums dtypes

In [234]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10956784 entries, 0 to 10956783
Data columns (total 11 columns):
 #   Column                                                                Dtype 
---  ------                                                                ----- 
 0   C/A                                                                   object
 1   UNIT                                                                  object
 2   SCP                                                                   object
 3   STATION                                                               object
 4   LINENAME                                                              object
 5   DIVISION                                                              object
 6   DATE                                                                  object
 7   TIME                                                                  object
 8   DESC                                                        

All columns except DATE and TIME are in the correct format. Let's unite that columns and change to datetime.

In [235]:

# snake_case column names
df.columns=df.columns.str.lower().str.replace(" ","")
# converting date to datetime format and dropping initial columns
df["datetime"]=df["date"]+" "+df["time"]
df.datetime=pd.to_datetime(df["datetime"])
df.drop(columns=["date","time"],inplace=True)
# This code creates multindex that will help group dataframe
# and calculate absolute values for entries and exits instead of relevant.
df['stat_id']=df["station"]+" "+df["scp"]+" "+df["c/a"]
df=df.set_index(["stat_id","datetime"]).sort_index()




Now features are fine

----

Some of the data from turnstiles is reversed in the dataset. That causes negative values. For that purpose I'll take abs of diff() values.This method gets grouped dataframe and returns absolute values for entries and exits instead of relevant.

In [236]:
def calc(temp):
    #after recieving subset, creating another column with absolute values of passengers.    
    temp['entries_abs']=temp['entries'].diff().abs()
    temp['exits_abs']=temp['exits'].diff().abs()   
    return temp

In [237]:
#groupping by first index (stat_id) and applying calc() function.
df=df.groupby(level=0).apply(lambda x: calc(x))

In [238]:
df

c/a  unit       scp    station  \
stat_id                 datetime                                               
1 AV 00-00-00 H007      2021-10-16 00:00:00  H007  R248  00-00-00       1 AV   
                        2021-10-16 04:00:00  H007  R248  00-00-00       1 AV   
                        2021-10-16 08:00:00  H007  R248  00-00-00       1 AV   
                        2021-10-16 12:00:00  H007  R248  00-00-00       1 AV   
                        2021-10-16 16:00:00  H007  R248  00-00-00       1 AV   
...                                           ...   ...       ...        ...   
ZEREGA AV 00-05-01 R419 2022-10-14 04:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-14 08:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-14 12:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-14 16:00:00  R419  R326  00-05-01  ZEREGA AV   
                        2022-10-14 20:00:00  R419  R326  00-05-01  ZEREGA AV   

                                            linename division     desc  \
stat_id                 datetime                                         
1 AV 00-00-00 H007      2021-10-16 00:00:00        L      BMT  REGULAR   
                        2021-10-16 04:00:00        L      BMT  REGULAR   
                        2021-10-16 08:00:00        L      BMT  REGULAR   
                        2021-10-16 12:00:00        L      BMT  REGULAR   
                        2021-10-16 16:00:00        L      BMT  REGULAR   
...                                              ...      ...      ...   
ZEREGA AV 00-05-01 R419 2022-10-14 04:00:00        6      IRT  REGULAR   
                        2022-10-14 08:00:00        6      IRT  REGULAR   
                        2022-10-14 12:00:00        6      IRT  REGULAR   
                        2022-10-14 16:00:00        6      IRT  REGULAR   
                        2022-10-14 20:00:00        6      IRT  REGULAR   

                                              entries     exits  entries_abs  \
stat_id                 datetime                                               
1 AV 00-00-00 H007      2021-10-16 00:00:00  15644134  18145813          NaN   
                        2021-10-16 04:00:00  15644151  18146053         17.0   
                        2021-10-16 08:00:00  15644173  18146229         22.0   
                        2021-10-16 12:00:00  15644247  18146605         74.0   
                        2021-10-16 16:00:00  15644355  18147155        108.0   
...                                               ...       ...          ...   
ZEREGA AV 00-05-01 R419 2022-10-14 04:00:00        39       289          0.0   
                        2022-10-14 08:00:00        39       289          0.0   
                        2022-10-14 12:00:00        39       289          0.0   
                        2022-10-14 16:00:00        39       289          0.0   
                        2022-10-14 20:00:00        39       289          0.0   

                                             exits_abs  
stat_id                 datetime                        
1 AV 00-00-00 H007      2021-10-16 00:00:00        NaN  
                        2021-10-16 04:00:00      240.0  
                        2021-10-16 08:00:00      176.0  
                        2021-10-16 12:00:00      376.0  
                        2021-10-16 16:00:00      550.0  
...                                                ...  
ZEREGA AV 00-05-01 R419 2022-10-14 04:00:00        0.0  
                        2022-10-14 08:00:00        0.0  
                        2022-10-14 12:00:00        0.0  
                        2022-10-14 16:00:00        0.0  
                        2022-10-14 20:00:00        0.0  

[10956784 rows x 11 columns]

Check for negative values

In [239]:
df[df["entries_abs"]<0]

,,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
stat_id,datetime,,,,,,,,,,,


In [240]:
df[df["exits_abs"]<0]

,,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
stat_id,datetime,,,,,,,,,,,


Checking and dropping all NaN values

In [241]:
df.isnull().sum()

c/a               0
unit              0
scp               0
station           0
linename          0
division          0
desc              0
entries           0
exits             0
entries_abs    5085
exits_abs      5085
dtype: int64

In [242]:
df.dropna(inplace=True)
df.isnull().sum()

c/a            0
unit           0
scp            0
station        0
linename       0
division       0
desc           0
entries        0
exits          0
entries_abs    0
exits_abs      0
dtype: int64

----

As you can see on the "entries" column, for the turnstile 01-05-00 at WORLD TRADE CTR Station, calculated number of entries was 2146956625, and in eight hours it changes to 8098. That seriously affects absolute value calculation.

In [245]:
#example of abnormal value
df.loc["WORLD TRADE CTR 01-03-04 N094"].iloc[1855:1860]

,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
datetime,,,,,,,,,,,
2022-08-21 08:00:00,N094,R029,01-03-04,WORLD TRADE CTR,ACE23,IND,REGULAR,527005,183608,3.000000e+00,4.0
2022-08-21 12:00:00,N094,R029,01-03-04,WORLD TRADE CTR,ACE23,IND,REGULAR,2146956625,183611,2.146430e+09,3.0
2022-08-21 16:00:00,N094,R029,01-03-04,WORLD TRADE CTR,ACE23,IND,REGULAR,527071,183618,2.146430e+09,7.0
2022-08-21 20:00:00,N094,R029,01-03-04,WORLD TRADE CTR,ACE23,IND,REGULAR,527116,183624,4.500000e+01,6.0
2022-08-22 00:00:00,N094,R029,01-03-04,WORLD TRADE CTR,ACE23,IND,REGULAR,527129,183626,1.300000e+01,2.0


Another example of incorrect data - some of the turnstile detectors were reseted. Since entries and exits columns show the change of amount of passengers, the main problem is between two rows with different calculation system. For example:

In [246]:
#example of turnstile reset
df.loc["125 ST 00-06-00 R258"].iloc[-52:-47]


,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs
datetime,,,,,,,,,,,
2022-10-05 04:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,275773227,324332636,227.0,442.0
2022-10-05 08:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,275773365,324332903,138.0,267.0
2022-10-06 12:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,275773530,324333223,165.0,320.0
2022-10-06 20:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,1325,254,275772205.0,324332969.0
2022-10-07 00:00:00,R258,R132,00-06-00,125 ST,456,IRT,REGULAR,1362,327,37.0,73.0


 So I'll change this incorrect values with the mean value of this exact station. The threshold to detect such fluctuations will be if that absolute value is greater than 15000. I follow the logic that it is phisically impossible to pass turnstile faster than for a second. For a period of 4 hours this gives us theoretical max amount of people at 4 * 60 * 60 = 14400.

First of all, I'll drop the index to find and replace these incorrect values with mean of their station respectively and the replace incorrect values with mean.

In [247]:
def fix_incorrect():
    '''Checks for all values larger than 15000 and replaces them with mean value for that station'''
    df.reset_index(inplace=True)
    for x,k in df.iterrows():
        #if absolute value of people passed through turnstile is more than 15000, replacing this value with mean.
        if df.loc[x,"entries_abs"]>15000:
            mean=df[(df["entries_abs"]<15000) & (df["station"]==k[5])]["entries_abs"].mean()
            mean=int(mean)    
            df.loc[x,"entries_abs"]=mean
    for x,k in df.iterrows():
    #if absolute value of people passed through turnstile is more than 15000, replacing this value with mean.
        if df.loc[x,"exits_abs"]>15000:
            mean=df[(df["exits_abs"]<15000) & (df["station"]==k[5])]["exits_abs"].mean()
            mean=int(mean)    
            df.loc[x,"exits_abs"]=mean

In [248]:
fix_incorrect()

Check for incorrect data.

In [249]:
df[df["entries_abs"]>15000]

,stat_id,datetime,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs


In [250]:
df[df["exits_abs"]>15000]

,stat_id,datetime,c/a,unit,scp,station,linename,division,desc,entries,exits,entries_abs,exits_abs


Everything is fine.

----

Now, let's change entries_abs and exits_abs to int and after that I'll drop: 
- entries and exits (we already used them)
- division - this feature contains information about station owner. This won't be useful for this project.
- c/a - this feature contains information about electrical station that supplies energy for current station. This won't be useful for this project either.
- desc - represent the "REGULAR" scheduled audit event. This won't be useful for this project.


In [251]:
def columns_drop():
    #changing columns type to int
    df["entries_abs"]=df["entries_abs"].astype(int)
    df["exits_abs"]=df["exits_abs"].astype(int)
    #droping columns
    df.drop(columns=["entries","exits","division","desc","stat_id"],inplace=True)
    #adding weekday column
    df['weekday'] = df['datetime'].dt.day_name()


In [252]:
columns_drop()

Saving file to csv.

In [253]:
df.to_csv(f'../data/df_cleaned_year.csv',index=False)
